In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
!pip install -r requirements.txt
import os

from src.preprocessing import DataPreprocessor
from src.model import Model
from src.gui import AnalysisGUI

You should consider upgrading via the 'C:\Users\csbro\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Load training and testing datasets
dataset = DataPreprocessor(
    filepath="data/Line Listing(dxjZy).xlsx",
    column_names=['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']
)
dataset.explode_column("Suspect Product Active Ingredients", ";")
medications = dataset.data["Suspect Product Active Ingredients"] # SAVED FOR LATER FOR MENUS IN GUI

print("Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nData Types:")
print(dataset.get_dataframe().dtypes)

dataset.get_dummies(["Suspect Product Active Ingredients"])
dataset.data.groupby("Case ID").max().reset_index()
dataset.drop_columns(["Case ID"])

dataset.ensure_numeric_column("Patient Age", decimal=False)

dataset.convert_nulls("Patient Weight", nulls=["Not Specified"], output="0 KG")
dataset.ensure_numeric_column("Patient Weight")

dataset.encode_column("Sex")
dataset.encode_column("Serious")

dataset.drop_all_nulls()

print("\nNew Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nDataset Head:")
print(dataset.get_dataframe().head())

x_train, x_test, y_train, y_test = dataset.get_standardised_train_test_split(
    dataset.data.columns, 
    y="Serious", 
    test_size=0.2, 
    random_state=42
)

Value Counts:
Serious
Serious        3199
Non-Serious      67
Name: count, dtype: int64

Data Types:
Case ID                                int64
Suspect Product Active Ingredients    object
Serious                               object
Sex                                   object
Patient Age                           object
Patient Weight                        object
dtype: object

New Value Counts:
Serious
1    3199
0      67
Name: count, dtype: int64

Dataset Head:
    Case ID  Serious  Sex  Patient Age  Patient Weight  .Alpha.-Tocopherol  \
0  24750965        1    1           75               0               False   
0  24750965        1    1           75               0               False   
1  24391183        1    0           73               0               False   
1  24391183        1    0           73               0               False   
1  24391183        1    0           73               0               False   

   .Alpha.-Tocopherol Acetate, Dl-  .Alpha.-Tocopherol, Dl

In [59]:
# Creating model
model_name = "default"

input_dim = len(x_train[0])
analysis_model = Model(input_dim)
if os.path.exists(f"models/{model_name}.pt"):
    analysis_model.load_model(model_name)
else:
    analysis_model.train_model(x_train, y_train)
    analysis_model.save_model(model_name)

analysis_model.evaluate_model(x_test, y_test)

Evaluation Loss: 0.01966577209532261


c:\Users\csbro\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([654])) that is different to the input size (torch.Size([654, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [60]:
# Show GUI
medications = medications.unique()

allergies = []

gui = AnalysisGUI(model=analysis_model, medications=medications, allergies=allergies)